**IMPORTACIÓN DE LIBRERIAS ÚTILES PARA TODO EL CICLO**

In [ ]:
#COMANDOS INICIALES
# pip install fastapi
# pip install fastapi uvicorn

In [4]:
import numpy as np
import pandas as pd
import psutil

from fastapi import FastAPI, HTTPException, Query, APIRouter

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

**Carga y Visualización de Datos**

In [5]:
# Link para API
app = FastAPI()    #http://192.168.1.10:8000

In [10]:
# Importamos el primer DF
df_items_games = pd.read_parquet('items_games.parquet')

# Visualización del DF importado
df_items_games

,user_id,item_id,game_name,playtime_forever,game_genre,year_release
0,76561197970982479,10,Counter-Strike,6,Action,2000
1,js41637,10,Counter-Strike,0,Action,2000
2,Riot-Punch,10,Counter-Strike,0,Action,2000
3,doctr,10,Counter-Strike,93,Action,2000
4,corrupted_soul,10,Counter-Strike,108,Action,2000
...,...,...,...,...,...,...
7190751,76561198071808318,444770,Mimpi Dreams,46,Indie,2016
7190752,76561198107283457,354280,ChaosTower,164,Action,2016
7190753,76561198107283457,354280,ChaosTower,164,Casual,2016
7190754,76561198107283457,354280,ChaosTower,164,Indie,2016


In [11]:
# Importamos el segundo DF
df_items_games_API = pd.read_parquet('items_games_API.parquet')

# Visualización del DF importado
df_items_games_API

,game_genre,year_release,playtime_forever,user_id
0,Action,2000,6,76561197970982479
1,Action,2000,0,js41637
2,Action,2000,0,Riot-Punch
3,Action,2000,93,doctr
4,Action,2000,108,corrupted_soul
...,...,...,...,...
7190751,Indie,2016,46,76561198071808318
7190752,Action,2016,164,76561198107283457
7190753,Casual,2016,164,76561198107283457
7190754,Indie,2016,164,76561198107283457


In [5]:
# Importamos el tercer DF
df_reviews_games_API = pd.read_parquet('reviews_games_API.parquet')

# Visualización del DF importado
df_reviews_games_API

,user_id,year_posted,item_id,recommend,analisis_sentimiento,game_genre,game_name,year_release
0,76561197970982479,2011,1250,1,2,Action,Killing Floor,2009
1,death-hunter,2015,1250,1,2,Action,Killing Floor,2009
2,DJKamBer,2013,1250,1,0,Action,Killing Floor,2009
3,diego9031,2015,1250,1,1,Action,Killing Floor,2009
4,76561198081962345,2014,1250,1,1,Action,Killing Floor,2009
...,...,...,...,...,...,...,...,...
99034,Gamer0009,2015,306040,1,2,Simulation,The Howler,2016
99035,Gamer0009,2015,306040,1,2,Strategy,The Howler,2016
99036,llDracuwulf,2015,307130,1,2,Action,Asteria,2014
99037,llDracuwulf,2015,307130,1,2,Adventure,Asteria,2014


In [6]:
# Importamos el cuarto DF
df_games_API = pd.read_parquet('games_API.parquet')

# Visualización del DF importado
df_games_API

,item_id,game_name,description
0,761140,Lost Summoner Kitty,"Action, Casual, Indie, Simulation, Strategy, S..."
1,643980,Ironbound,"Free to Play, Indie, RPG, Strategy, Free to Pl..."
2,670290,Real Pool 3D - Poolians,"Casual, Free to Play, Indie, Simulation, Sport..."
3,767400,弹炸人2222,"Action, Adventure, Casual, Action, Adventure, ..."
4,772540,Battle Royale Trainer,"Action, Adventure, Simulation, Action, Adventu..."
...,...,...,...
22525,745400,Kebab it Up!,"Action, Adventure, Casual, Indie, Action, Indi..."
22526,773640,Colony On Mars,"Casual, Indie, Simulation, Strategy, Strategy,..."
22527,733530,LOGistICAL: South Africa,"Casual, Indie, Strategy, Strategy, Indie, Casu..."
22528,610660,Russian Roads,"Indie, Racing, Simulation, Indie, Simulation, ..."


**GENERACIÓN DE APIs**

Función PlayTimeGenre - Total de horas jugadas en determinado género por año.

In [9]:
# Creamos una API para la Función PlayTimeGenre
@app.get("/PlayTimeGenre/{genre}")
def PlayTimeGenre(genre: str):
    # Filtramos el dataframe 'df_items_games_API' respecto al parámetro 'genre'
    genre_data = df_items_games_API[df_items_games_API['game_genre'] == genre]

    if genre_data.empty:
        return {"genre": genre, "most_played_year": None}

    # Agrupamos respecto a los años y suma de tiempo de juego
    year_playtime = genre_data.groupby('year_release')['playtime_forever'].sum()
    most_played_year = year_playtime.idxmax()

    return {"Total de horas jugadas en determinado género por año:"}, {"Género": genre}, {"Año con mayor horas jugadas": int(most_played_year)}

In [10]:
# Visualizamos el resultado de la consulta a la API
PlayTimeGenre('Casual')

({'Total de horas jugadas en determinado género por año:'},
 {'Género': 'Casual'},
 {'Año con mayor horas jugadas': 2017})

Función UserForGenre - Total de horas jugadas por usuario en determinado género.

In [43]:
# Creamos una API para la Función UserForGenre
@app.get("/UsersForGenre/{genre}")
def UserForGenre(genre: str):
    # Filtramos el dataframe 'df_items_games_API' respecto al parámetro 'genre'
    genre_data = df_items_games_API[df_items_games_API['game_genre']== genre]

    # Agrupamos el dataframe anterior por 'user_id', por la suma del tiempo de juego y ordenamos
    group = genre_data.groupby('user_id')['playtime_forever'].sum().sort_values(ascending=False)

    # Reindexamos a valor máximo a [0]
    user = group.index[0]

    # Tomamos las filas del dataframe que contengan 'user_id'
    user_genre = genre_data[genre_data['user_id']==user]

    # Agrupamos respecto a los años y suma de tiempo de juego
    played_hours = round(user_genre.groupby('year_release')['playtime_forever'].sum(), 3)

    # Guardamos la serie 'played_hours' en una lista para su uso posterior
    total_hours_played = [f'Año: {int(anio)}, Horas: {horas}' for anio, horas in played_hours.items()]

    return {f"Usuario con más horas jugadas para género, {genre}": user, "Horas jugadas": total_hours_played}

In [44]:
# Visualizamos el resultado de la consulta a la API
UserForGenre('Action')

{'Usuario con más horas jugadas para género, Action': 'REBAS_AS_F-T',
 'Horas jugadas': ['Año: 1996, Horas: 0',
  'Año: 1998, Horas: 1',
  'Año: 1999, Horas: 224',
  'Año: 2000, Horas: 0',
  'Año: 2001, Horas: 11',
  'Año: 2002, Horas: 1',
  'Año: 2003, Horas: 1863',
  'Año: 2004, Horas: 2115',
  'Año: 2005, Horas: 3293',
  'Año: 2006, Horas: 138',
  'Año: 2007, Horas: 375',
  'Año: 2008, Horas: 2573',
  'Año: 2009, Horas: 7926',
  'Año: 2010, Horas: 4460',
  'Año: 2011, Horas: 37588',
  'Año: 2012, Horas: 50635',
  'Año: 2013, Horas: 95383',
  'Año: 2014, Horas: 158114',
  'Año: 2015, Horas: 156262',
  'Año: 2016, Horas: 136809',
  'Año: 2017, Horas: 1990']}

Función UsersRecommend - Usuarios Recomiendan

In [35]:
# Creamos una API para la Función UsersRecommend
@app.get("/UsersRecommend/{year}")
def UsersRecommend(year: int):
    # Filtramos el dataframe 'df_reviews_games_API' en base al 'year_posted' y al 'analisis_sentimiento' positivo
    filter = df_reviews_games_API[(df_reviews_games_API['recommend'] == True) & (df_reviews_games_API['analisis_sentimiento'] >= 1) & ((df_reviews_games_API['year_posted']) == year)]

    if filter.empty:
        return {"error": "No recommended games found for the given year"}

    # Agrupamos por 'game_name' y calculamos el número de recomendaciones
    recommendations = filter.groupby('game_name')['recommend'].sum()

    if recommendations.empty:
        return {"error": "No recommended games found for the given year"}

    # Ordenamos los juegos por recomendaciones positivas
    order = recommendations.sort_values(ascending=False)

    # Tomamos el top 3
    top_3 = order.head(3)

    # Creamos el resultado por posiciones
    result = [{"Top 3 Juegos Recomendados por Usuarios:"}, {"1st Place": top_3.index[0]}, {"2nd Place": top_3.index[1]}, {"3rd Place": top_3.index[2]}]

    return result

In [36]:
# Visualizamos el resultado de la consulta a la API
UsersRecommend(2013)

[{'Top 3 Juegos Recomendados por Usuarios'},
 {'1st Place': 'Team Fortress 2'},
 {'2nd Place': "Garry's Mod"},
 {'3rd Place': 'Terraria'}]

Función UsersNotRecommend - Usuarios NO Recomiendan

In [33]:
# Creamos una API para la Función UsersNotRecommend
@app.get("/UsersNotRecommend/{year}")
def UsersNotRecommend(year: int):
    # Filtramos el dataframe 'df_reviews_games_API' en base al 'year_posted' y al 'analisis_sentimiento' negativo
    filter = df_reviews_games_API[(df_reviews_games_API['recommend'] == False) & (df_reviews_games_API['analisis_sentimiento'] == 0) & ((df_reviews_games_API['year_posted']) == year)]

    if filter.empty:
        return {"error": "No least recommended games found for the given year"}

    # Agrupamos por 'game_name' y calculamos el número de recomendaciones
    not_recommendations = filter.groupby('game_name')['recommend'].count()

    if not_recommendations.empty:
        return {"error": "No least recommended games found for the given year"}

   # Ordenamos los juegos por recomendaciones negativas
    order = not_recommendations.sort_values(ascending=True)

     # Tomamos el top 3
    top_3 = order.head(3)

    # Creamos el resultado por posiciones
    result = [{"Top 3 Juegos Recomendados NO por Usuarios:"}, {"1st Place": top_3.index[0]}, {"2nd Place": top_3.index[1]}, {"3rd Place": top_3.index[2]}]

    return result

In [34]:
# Visualizamos el resultado de la consulta a la API
UsersNotRecommend(2013)

[{'Top 3 Juegos Recomendados NO por Usuarios:'},
 {'1st Place': 'Worms Clan Wars'},
 {'2nd Place': 'F.E.A.R. 3'},
 {'3rd Place': 'Left 4 Dead 2'}]

Función sentiment_analysis - Análisis de Sentimiento

In [47]:
# Creamos una API para el Análisis de Sentimiento
@app.get("/sentiment_analysis/{year}")
def sentiment_analysis(year: int):
    # Filtramos el dataframe 'df_reviews_games_API' respecto al parámetro 'year'
    filter = df_reviews_games_API[(df_reviews_games_API['year_release']) == year]

    if filter.empty:
        return {"error": "No data found for the given year"}

    # Agrupamos por el 'analisis_sentimiento' y calculamos la suma de cada sentimiento
    group_sentiments = filter.groupby(['analisis_sentimiento'])['user_id'].nunique().reset_index()

    # Renombramos alguna columna y mapearemos los valores de sentimientos por niveles 
    group_sentiments.columns = ['Sentiment', 'Count']
    sentiment_labels = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    group_sentiments['Sentiment'] = group_sentiments['Sentiment'].map(sentiment_labels)

    # Convertimos el DF a un diccionario
    sentiment_counts = group_sentiments.set_index('Sentiment')['Count'].to_dict()

    return sentiment_counts

In [49]:
# Visualizamos el resultado de la consulta a la API
sentiment_analysis(2015)

{'Negative': 795, 'Neutral': 751, 'Positive': 2462}

**API de Aprendizaje Automático**

Función Recomendacion_Juego - Juego Recomendado por 'itemID'

In [18]:
# Creamos una instancia con: CountVectorizer
vector = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Dividimos cada cadena de descripción en palabras individuales y creamos una matriz de conteo la cual mostrará cada género de cada videojuego.
matriz_description = vector.fit_transform(df_games_API['description'])

c:\Program Files\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [19]:
# Creamos una API para la recomendación de Juegos en base a otros Juegos, la misma tendrá ML
@app.get("/recommend_games_by_games/{game_id}")
def recommend_games_by_games(game_id: int):
    '''
    Mediante esta función podremos observar la recomendación mediante el uso de ML en base a un juego ingresado.
    '''
    # Se ingresa el item_id y retornamos una lista con 5 juegos recomendados similares al ingresado (title)

    if game_id not in df_games_API['item_id'].values:
        return 'El ID no existe, intente con otro'
    else:
        index = df_games_API.index[df_games_API['item_id']==game_id][0]
        description_index = matriz_description[index]

        ''' Calculamos la similitud coseno entre la descripción de entrada y la descripción de las demás filas: cosine_similarity.
            Obtenemos los índices de las mayores similitudes mediante el método argsort() y las similitudes ordenadas de manera descendente.
            Tomamos los índices del 1 al 6 [0, 1:6] ya que el índice 0 es el mismo índice de entrada.
        '''
        
        max_index = np.argsort(-cosine_similarity(description_index, matriz_description))[0, 1:6]

        recomendations = []
        for i in max_index:
            recomendations.append(df_games_API['game_name'][i])
        
        return recomendations

In [20]:
# Visualizamos el resultado de la consulta a la API
recommend_games_by_games(761140)

['Trivia Vault: Super Heroes Trivia 2',
 'Crankies Workshop: Grizzbot Assembly',
 'Trivia Vault: Science &amp; History Trivia 2',
 "Trivia Vault: 1980's Trivia 2",
 'Trivia Vault: Classic Rock Trivia']